# dien data preprocess

In [4]:
import os
import tensorflow as tf
import numpy as np
import sys
import random
import time
import pickle

## process_data.py

### 1.process meta

{'asin': '0001047566', 'salesRank': {'Books': 3628249}, 'imUrl': 'http://ecx.images-amazon.com/images/I/51FWARNTHCL.jpg', 'categories': [['Books']], 'title': 'Hand in Glove'}

{'asin': '0001053396', 
'description': 'Writing poetry has always been a way to express what is going on in the world around us. Unfortunately, sometimes what is around us is war. Acclaimed British actor Paul McGann reads poems that date from the eighteenth century to modern times in this moving audio anthology. McGann is originally from Liverpool, and he still retains a slight Northern accent. Its lilting cadence blends well with the rhythm and worldliness of these powerful poems. Although some of the pieces are well known, others are less so, and this makes the collection even more intriguing for lovers of literature.  R.A.P. &copy; AudioFile 2001, Portland, Maine--Copyright &#169; AudioFile, Portland, Maine', 
'title': 'War Poems: An Anthology of Poetry from the 18th Cantury to the Present Day', 'price': 17.99, 
'imUrl': 'http://ecx.images-amazon.com/images/I/51WTKK4V2DL.jpg', 
'salesRank': {'Books': 12249714}, 'categories': [['Books']]}

{'asin': '0000401048', 'title': "The rogue of publishers' row;: Confessions of a publisher (A Banner Book)", 'imUrl': 'http://ecx.images-amazon.com/images/I/41bchvIfgaL.jpg', 'related': {'also_viewed': ['068240103X']}, 'salesRank': {'Books': 6448843}, 'categories': [['Books']]}

{'asin': '0001019880', 'title': "Classic Soul Winner's New Testament Bible", 'price': 5.39, 'imUrl': 'http://ecx.images-amazon.com/images/I/61LcHUdvS1L.jpg', 'related': {'also_viewed': ['B003HMB5FC', '0834004593'], 'buy_after_viewing': ['031095360X', '0834004577']}, 'salesRank': {'Books': 9589258}, 'categories': [['Books']]}

{'asin': '0001048813', 'imUrl': 'http://ecx.images-amazon.com/images/I/41k5u0lr6NL.jpg', 'categories': [['Books']], 'title': 'Archer Christmas 4 Tape Pack'}

{'asin': '0001148427', 'salesRank': {'Books': 5806769}, 'imUrl': 'http://ecx.images-amazon.com/images/I/41tN4KuOXkL.jpg', 'categories': [['Books']], 'title': 'Sonatas - For Piano'}


In [3]:
meta_file = 'D:/workspace/data/dien/meta_books1000.json'

In [19]:
def process_meta(file):
    fi = open(file, "r")
    fo = open("item-info", "w")
    for line in fi:
        print(line)
        obj = eval(line)
        print('-'*20)
        print(obj)
        print('*'*20)
        cat = obj["categories"][0][-1]
        outline = obj["asin"] + "\t" + cat+ "\n"
        print(outline)
        fo.write(outline)

In [20]:
process_meta(meta_file)  # itemID cate_str

{'asin': '0001048791', 'salesRank': {'Books': 6334800}, 'imUrl': 'http://ecx.images-amazon.com/images/I/51MKP0T4DBL.jpg', 'categories': [['Books']], 'title': 'The Crucible: Performed by Stuart Pankin, Jerome Dempsey &amp; Cast'}

--------------------
{'asin': '0001048791', 'salesRank': {'Books': 6334800}, 'imUrl': 'http://ecx.images-amazon.com/images/I/51MKP0T4DBL.jpg', 'categories': [['Books']], 'title': 'The Crucible: Performed by Stuart Pankin, Jerome Dempsey &amp; Cast'}
********************
0001048791	Books

{'asin': '0001048775', 'description': 'William Shakespeare is widely regarded as the greatest playwright the world has seen. He produced an astonishing amount of work; 37 plays, 154 sonnets, and 5 poems. He died on 23rd April 1616, aged 52, and was buried in the Holy Trinity Church, Stratford.', 'title': 'Measure for Measure: Complete &amp; Unabridged', 'imUrl': 'http://ecx.images-amazon.com/images/I/5166EBHDQYL.jpg', 'salesRank': {'Books': 13243226}, 'categories': [['Books']]

### 2.process reviews

{"reviewerID": "ACNGUPJ3A3TM9", "asin": "0000013714", "reviewerName": "GCM", "helpful": [0, 0], "reviewText": "We use this type of hymnal at church.  I was looking for the same one; however, this wasn't it. It is a good hymnal, but there isn't enough information to find the version I need.", "overall": 4.0, "summary": "Nice Hymnal", "unixReviewTime": 1386028800, "reviewTime": "12 3, 2013"}

{"reviewerID": "A3BED5QFJWK88M", "asin": "0000013714", "reviewerName": "jimedd", "helpful": [0, 0], "reviewText": "Heavenly Highway HymnsI ordered this hymnal because I learned to read shaped note music when I was a teenager. I play piano but do not sing. I am 85 years old. This hymnal has mostof the songs I have learned over the years. It was exactly what I wanted and needed.It was in good condition and the price was right. I purchased this book from Amazon.", "overall": 4.0, "summary": "Heavenly Highway Hymns", "unixReviewTime": 1350345600, "reviewTime": "10 16, 2012"}

{"reviewerID": "A2SUAM1J3GNN3B", "asin": "0000013714", "reviewerName": "J. McDonald", "helpful": [2, 3], "reviewText": "I bought this for my husband who plays the piano.  He is having a wonderful time playing these old hymns.  The music  is at times hard to read because we think the book was published for singing from more than playing from.  Great purchase though!", "overall": 5.0, "summary": "Heavenly Highway Hymns", "unixReviewTime": 1252800000, "reviewTime": "09 13, 2009"}

{"reviewerID": "APOZ15IEYQRRR", "asin": "0000013714", "reviewerName": "maewest64", "helpful": [0, 0], "reviewText": "This is a large size hymn book which is great to be able to see the songs, notes, etc.  Quality was great, item was new!", "overall": 5.0, "summary": "Awesome Hymn Book", "unixReviewTime": 1362787200, "reviewTime": "03 9, 2013"}


In [13]:
reviews_file = 'D:/workspace/data/dien/reviews1000.json'

In [21]:
def process_reviews(file):
    fi = open(file, "r")
    user_map = {}
    fo = open("reviews-info", "w")
    for line in fi:
        obj = eval(line)
        userID = obj["reviewerID"]
        itemID = obj["asin"]
        rating = obj["overall"]
        time = obj["unixReviewTime"]
        outline = userID + "\t" + itemID + "\t" + str(rating) + "\t" + str(time)+"\n"
        print(outline)
        fo.write(outline)

In [22]:
process_reviews(reviews_file)  # userID itemID rating time

AH2L9G3DQHHAJ	0000000116	4.0	1019865600

A2IIIDRK3PRRZY	0000000116	1.0	1395619200

A1TADCM7YWPQ8M	0000000868	4.0	1031702400

AWGH7V0BDOJKB	0000013714	4.0	1383177600

A3UTQPQPM4TQO0	0000013714	5.0	1374883200

A8ZS0I5L5V31B	0000013714	5.0	1393632000

ACNGUPJ3A3TM9	0000013714	4.0	1386028800

A3BED5QFJWK88M	0000013714	4.0	1350345600

A2SUAM1J3GNN3B	0000013714	5.0	1252800000

APOZ15IEYQRRR	0000013714	5.0	1362787200

AYEDW3BFK53XK	0000013714	5.0	1325462400

A1KLCGLCXYP1U1	0000013714	3.0	1376092800

A37W6POFWIVG13	0000013714	5.0	1316131200

A2EIPZNHAEXZHJ	0000013714	4.0	1325030400

A1VAFVJFT58YI3	0000013714	5.0	1384300800

A9KTKY6BUR8U6	0000013714	1.0	1357516800

A2742OG8PK8KU6	0000013714	5.0	1358380800

A38AAPXSJN4C5G	0000015393	4.0	1239494400

A14A5Q8VJK5NLR	0000029831	4.0	1393286400

A3W2PX96K1BA3M	0000029831	5.0	1393200000

A2GKR2Q7MD8DG4	0000029831	5.0	1393286400

A1MC4E00RO5E9T	0000029831	5.0	1393200000

A23PISU0ZLW71C	0000029831	5.0	1393200000

A1KP12IVP6LAGL	0000038504	5.0	1232755200



A372BPDH0UAGNR	0001056107	5.0	952128000

A1AF30H2MPOO9	0001056530	4.0	966816000

A3QIVQ3SKET6TW	0001056530	4.0	964051200

AONMTGS00YTK7	0001057138	5.0	1314662400

ADDB0Y73L2CHU	0001057146	3.0	994118400

ADDB0Y73L2CHU	0001057170	3.0	991612800

A1CUCKRYAXFH1C	0001057324	5.0	1114300800

A2T5RBN49FVXZX	0001057324	4.0	1175040000

A2TAEHWKIREZP9	0001061100	5.0	1360281600

A1H01R0MEDEUC6	0001061100	5.0	1384819200

A373Y963BD0M2L	0001061127	5.0	1224806400

A142EFCHX4ZJ31	0001061127	5.0	1249430400

A25DDPF5TGEJT9	0001061127	5.0	1402099200

A1MV6SMIRQ5YU7	0001062395	5.0	1371340800

A12UFE6RPVTZJT	0001064487	4.0	1360281600

A3GFXEFR8FDX6P	0001064487	5.0	1309046400

A17K364R0ETIJJ	0001064487	5.0	1355961600

A1V9HZP9ONKV78	0001064487	5.0	1367280000

A53ZWHWGLPZQV	0001072986	5.0	1376611200

A3F1VVP8RKWFC	0001127748	5.0	1385856000

A1VBWYIOZQQIDQ	0001148427	5.0	1397779200

A15E0ZBE01DCDH	0001203010	5.0	1382745600

A2JS5MT00ZNGA2	0001203088	5.0	1357257600

A3ECAVKWPB26HV	0001360000	5.0	1196035200

A1

In [33]:
def manual_join(rev_file="reviews-info",meta_file="item-info",fout="jointed-new"):
    f_rev = open(rev_file, "r")
    user_map = {}
    item_list = []
    for line in f_rev:
        line = line.strip()
        items = line.split("\t")
        #loctime = time.localtime(float(items[-1]))
        #items[-1] = time.strftime('%Y-%m-%d', loctime)
        if items[0] not in user_map:
            user_map[items[0]]= []
        user_map[items[0]].append(("\t".join(items), float(items[-1])))  # {userID:[items,time_float]}
        item_list.append(items[1])  # [itemID]
        
    print('\n------user_map------\n',user_map)
    print('\n------item_list-----\n',item_list)
    
    f_meta = open(meta_file, "r")
    meta_map = {}
    for line in f_meta:
        arr = line.strip().split("\t")
        if arr[0] not in meta_map:
            meta_map[arr[0]] = arr[1]  # {itemID:cate_str}
            arr = line.strip().split("\t")
            
    print('\n========meta_map======\n',meta_map)
            
    fo = open(fout, "w")
    for key in user_map:
        sorted_user_bh = sorted(user_map[key], key=lambda x:x[1])
        print('key:{},sorted_user_bh:'.format(key),sorted_user_bh)
        for line, t in sorted_user_bh:  # items, time_float
            items = line.split("\t")
            asin = items[1]  # itemID
            j = 0
            while True:
                asin_neg_index = random.randint(0, len(item_list) - 1)
                asin_neg = item_list[asin_neg_index]  # 随机抽一个itemID作为负例，可以改成从meta_map.keys()里面抽取负例itemID，后边不报错
                if asin_neg == asin:
                    continue   # 与当前itemID重复则重新抽取负例itemID
                items[1] = asin_neg  # 将当前样本itemID替换为负例itemID且加上cate信息
                try:  # 我加的，因为数据不全，防止报错
                    outline1 = "0" + "\t" + "\t".join(items) + "\t" + meta_map[asin_neg]+"\n"
                except:
                    continue
                print('   outline1：',outline1)
                fo.write(outline1)  # 将替换过itemID的负样本的样本记录
                j += 1
                if j == 1:             #negative sampling frequency，j==1，则每个样本取一个负例
                    break
            if asin in meta_map:
                outline2 = "1" + "\t" + line + "\t" + meta_map[asin]+"\n"  # itemID有记录的正例，加上cate写入文件
                print('in outline2：',outline2)
                fo.write(outline2)
            else:
                outline2 = "1" + "\t" + line + "\t" + "default_cat"+"\n"  # 没有分类记录的itemID，加上默认cate写入文件
                print('out outline2：',outline2)
                fo.write(outline2)

In [34]:
manual_join(rev_file="reviews-info",meta_file="item-info",fout="jointed-new")  # label,userID,itemID,rating,time,cate


------user_map------
 {'AH2L9G3DQHHAJ': [('AH2L9G3DQHHAJ\t0000000116\t4.0\t1019865600', 1019865600.0)], 'A2IIIDRK3PRRZY': [('A2IIIDRK3PRRZY\t0000000116\t1.0\t1395619200', 1395619200.0)], 'A1TADCM7YWPQ8M': [('A1TADCM7YWPQ8M\t0000000868\t4.0\t1031702400', 1031702400.0)], 'AWGH7V0BDOJKB': [('AWGH7V0BDOJKB\t0000013714\t4.0\t1383177600', 1383177600.0)], 'A3UTQPQPM4TQO0': [('A3UTQPQPM4TQO0\t0000013714\t5.0\t1374883200', 1374883200.0)], 'A8ZS0I5L5V31B': [('A8ZS0I5L5V31B\t0000013714\t5.0\t1393632000', 1393632000.0)], 'ACNGUPJ3A3TM9': [('ACNGUPJ3A3TM9\t0000013714\t4.0\t1386028800', 1386028800.0)], 'A3BED5QFJWK88M': [('A3BED5QFJWK88M\t0000013714\t4.0\t1350345600', 1350345600.0)], 'A2SUAM1J3GNN3B': [('A2SUAM1J3GNN3B\t0000013714\t5.0\t1252800000', 1252800000.0)], 'APOZ15IEYQRRR': [('APOZ15IEYQRRR\t0000013714\t5.0\t1362787200', 1362787200.0)], 'AYEDW3BFK53XK': [('AYEDW3BFK53XK\t0000013714\t5.0\t1325462400', 1325462400.0)], 'A1KLCGLCXYP1U1': [('A1KLCGLCXYP1U1\t0000013714\t3.0\t1376092800', 13760928

in outline2： 1	A2ZZHMT58ZMVCZ	000100039X	5.0	1137456000	Books

key:A5K32C2QZN3WL,sorted_user_bh: [('A5K32C2QZN3WL\t000100039X\t4.0\t1100649600', 1100649600.0)]
   outline1： 0	A5K32C2QZN3WL	0000202010	4.0	1100649600	Books

in outline2： 1	A5K32C2QZN3WL	000100039X	4.0	1100649600	Books

key:A3BWLXLEOHMD01,sorted_user_bh: [('A3BWLXLEOHMD01\t000100039X\t5.0\t1246320000', 1246320000.0)]
   outline1： 0	A3BWLXLEOHMD01	000171287X	5.0	1246320000	Books

in outline2： 1	A3BWLXLEOHMD01	000100039X	5.0	1246320000	Books

key:A3S7ZMG2KGJ5BA,sorted_user_bh: [('A3S7ZMG2KGJ5BA\t000100039X\t5.0\t1369440000', 1369440000.0)]
   outline1： 0	A3S7ZMG2KGJ5BA	0000013714	5.0	1369440000	Books

in outline2： 1	A3S7ZMG2KGJ5BA	000100039X	5.0	1369440000	Books

key:AQK9X858KUY9U,sorted_user_bh: [('AQK9X858KUY9U\t000100039X\t4.0\t1267056000', 1267056000.0)]
   outline1： 0	AQK9X858KUY9U	0001018043	4.0	1267056000	Books

in outline2： 1	AQK9X858KUY9U	000100039X	4.0	1267056000	Books

key:A1VSHT4FFP768T,sorted_user_bh: [('A1VSHT4

in outline2： 1	A1UF72FGY08K6V	000102521X	5.0	1068508800	Books

key:A39J92OJUYHVNK,sorted_user_bh: [('A39J92OJUYHVNK\t000102521X\t4.0\t1033516800', 1033516800.0), ('A39J92OJUYHVNK\t0001857029\t5.0\t1052438400', 1052438400.0)]
   outline1： 0	A39J92OJUYHVNK	000100039X	4.0	1033516800	Books

in outline2： 1	A39J92OJUYHVNK	000102521X	4.0	1033516800	Books

   outline1： 0	A39J92OJUYHVNK	000100039X	5.0	1052438400	Books

out outline2： 1	A39J92OJUYHVNK	0001857029	5.0	1052438400	default_cat

key:A1NA603N9BJHLU,sorted_user_bh: [('A1NA603N9BJHLU\t0001026038\t5.0\t1296172800', 1296172800.0)]
   outline1： 0	A1NA603N9BJHLU	0001055178	5.0	1296172800	Books

out outline2： 1	A1NA603N9BJHLU	0001026038	5.0	1296172800	default_cat

key:A319KYEIAZ3SON,sorted_user_bh: [('A319KYEIAZ3SON\t0001035649\t5.0\t1125360000', 1125360000.0)]
   outline1： 0	A319KYEIAZ3SON	000100039X	5.0	1125360000	Books

in outline2： 1	A319KYEIAZ3SON	0001035649	5.0	1125360000	Books

key:ANIMV3SPDD8SH,sorted_user_bh: [('ANIMV3SPDD8SH\t0001042

key:A2ZS91Z7MVJR92,sorted_user_bh: [('A2ZS91Z7MVJR92\t000171287X\t5.0\t1368316800', 1368316800.0)]
   outline1： 0	A2ZS91Z7MVJR92	000100039X	5.0	1368316800	Books

in outline2： 1	A2ZS91Z7MVJR92	000171287X	5.0	1368316800	Books

key:A1PKXLGXSUTNQP,sorted_user_bh: [('A1PKXLGXSUTNQP\t000171287X\t5.0\t1391558400', 1391558400.0)]
   outline1： 0	A1PKXLGXSUTNQP	0000013714	5.0	1391558400	Books

in outline2： 1	A1PKXLGXSUTNQP	000171287X	5.0	1391558400	Books

key:A2YNQXG0ERQUD3,sorted_user_bh: [('A2YNQXG0ERQUD3\t000171287X\t5.0\t1385683200', 1385683200.0)]
   outline1： 0	A2YNQXG0ERQUD3	000100039X	5.0	1385683200	Books

in outline2： 1	A2YNQXG0ERQUD3	000171287X	5.0	1385683200	Books

key:A2WHGYP1X72H4E,sorted_user_bh: [('A2WHGYP1X72H4E\t000171287X\t5.0\t1354752000', 1354752000.0)]
   outline1： 0	A2WHGYP1X72H4E	000100039X	5.0	1354752000	Books

in outline2： 1	A2WHGYP1X72H4E	000171287X	5.0	1354752000	Books

key:A179R0UL62Q36Z,sorted_user_bh: [('A179R0UL62Q36Z\t000171287X\t5.0\t1345680000', 1345680000.0)]
 

In [39]:
def split_test(fin="jointed-new",fout="jointed-new-split-info"):
    fi = open(fin, "r")  # # label,userID,itemID,rating,time,cate
    fo = open(fout, "w")
    user_count = {}
    for line in fi:
        line = line.strip()
        user = line.split("\t")[1]
        if user not in user_count:
            user_count[user] = 0
        user_count[user] += 1  # 统计每个userID有几个样本
    fi.seek(0)
    i = 0  # 记录每个userID录入了几个样本
    last_user = "A26ZDKC53OP6JD"  # 上一个userID
    for line in fi:
        line = line.strip()
        user = line.split("\t")[1]
        if user == last_user:
            if i < user_count[user] - 2:  # 1 + negative samples，如果改变负样本抽样频率，这里也要改变
                outline = "20180118" + "\t" + line+"\n"
                print('user:{},outline:'.format(user),outline)
                fo.write(outline)
            else:
                outline = "20190119" + "\t" + line+"\n"
                print('user:{},outline:'.format(user),outline)
                fo.write(outline)
        else:
            last_user = user
            i = 0
            if i < user_count[user] - 2:
                outline = "20180118" + "\t" + line+"\n"
                print('user:{},outline:'.format(user),outline)
                fo.write(outline)
            else:
                outline = "20190119" + "\t" + line+"\n"
                print('user:{},outline:'.format(user),outline)
                fo.write(outline)
        i += 1

In [40]:
split_test()  # 真time，label，userID, itemID, rating, time, cate_str

user:AH2L9G3DQHHAJ,outline: 20190119	0	AH2L9G3DQHHAJ	000100039X	4.0	1019865600	Books

user:AH2L9G3DQHHAJ,outline: 20190119	1	AH2L9G3DQHHAJ	0000000116	4.0	1019865600	Books

user:A2IIIDRK3PRRZY,outline: 20190119	0	A2IIIDRK3PRRZY	0001360000	1.0	1395619200	Books

user:A2IIIDRK3PRRZY,outline: 20190119	1	A2IIIDRK3PRRZY	0000000116	1.0	1395619200	Books

user:A1TADCM7YWPQ8M,outline: 20190119	0	A1TADCM7YWPQ8M	000100039X	4.0	1031702400	Books

user:A1TADCM7YWPQ8M,outline: 20190119	1	A1TADCM7YWPQ8M	0000000868	4.0	1031702400	Books

user:AWGH7V0BDOJKB,outline: 20190119	0	AWGH7V0BDOJKB	000100039X	4.0	1383177600	Books

user:AWGH7V0BDOJKB,outline: 20190119	1	AWGH7V0BDOJKB	0000013714	4.0	1383177600	Books

user:A3UTQPQPM4TQO0,outline: 20190119	0	A3UTQPQPM4TQO0	000100039X	5.0	1374883200	Books

user:A3UTQPQPM4TQO0,outline: 20190119	1	A3UTQPQPM4TQO0	0000013714	5.0	1374883200	Books

user:A8ZS0I5L5V31B,outline: 20190119	0	A8ZS0I5L5V31B	000100039X	5.0	1393632000	Books

user:A8ZS0I5L5V31B,outline: 20190119	1	A8Z


user:A2Q4HZUGLXP6F1,outline: 20190119	1	A2Q4HZUGLXP6F1	000100039X	5.0	1381190400	Books

user:A3SGEA0N8OU613,outline: 20190119	0	A3SGEA0N8OU613	000171287X	4.0	1289347200	Books

user:A3SGEA0N8OU613,outline: 20190119	1	A3SGEA0N8OU613	000100039X	4.0	1289347200	Books

user:A1NGIUZJ9KFRV3,outline: 20190119	0	A1NGIUZJ9KFRV3	000171287X	5.0	1390953600	Books

user:A1NGIUZJ9KFRV3,outline: 20190119	1	A1NGIUZJ9KFRV3	000100039X	5.0	1390953600	Books

user:AUGZ7EUD67ROM,outline: 20190119	0	AUGZ7EUD67ROM	0001714538	5.0	1210032000	Books

user:AUGZ7EUD67ROM,outline: 20190119	1	AUGZ7EUD67ROM	000100039X	5.0	1210032000	Books

user:A3AKZ17Y3FK7QW,outline: 20190119	0	A3AKZ17Y3FK7QW	000171287X	3.0	1373932800	Books

user:A3AKZ17Y3FK7QW,outline: 20190119	1	A3AKZ17Y3FK7QW	000100039X	3.0	1373932800	Books

user:A2EPG54INKRW25,outline: 20190119	0	A2EPG54INKRW25	0001361155	5.0	1236384000	Books

user:A2EPG54INKRW25,outline: 20190119	1	A2EPG54INKRW25	000100039X	5.0	1236384000	Books

user:A1DW9QEARBGRFO,outline: 201901

## local_aggretor.py

In [ ]:
# 通过作者提供的处理后数据直接查看


In [42]:
fin = open("jointed-new-split-info", "r")  # # 真time，label，userID, itemID, rating, time, cate_str
ftrain = open("local_train", "w")
ftest = open("local_test", "w")

last_user = "0"
common_fea = ""


In [44]:
line_idx = 0
for line in fin:
    items = line.strip().split("\t")
    ds = items[0]  # 真time
    clk = int(items[1])  # label
    user = items[2]  # userid
    movie_id = items[3]  # itemid
    dt = items[5]  # time
    cat1 = items[6]  # cate_str

    if ds=="20180118":
        fo = ftrain
    else:
        fo = ftest
    if user != last_user:  # 每个用户单独记录
        movie_id_list = []
        cate1_list = []
        #print >> fo, items[1] + "\t" + user + "\t" + movie_id + "\t" + cat1 +"\t" + "" + "\t" + "" 
    else:
        history_clk_num = len(movie_id_list)
        cat_str = ""
        mid_str = ""
        for c1 in cate1_list:
            cat_str += c1 + ""  # 把个人的cate串起来
        for mid in movie_id_list:
            mid_str += mid + ""  # 把个人的itemid串起来
        if len(cat_str) > 0: cat_str = cat_str[:-1]
        if len(mid_str) > 0: mid_str = mid_str[:-1]
        if history_clk_num >= 1:    # 8 is the average length of user behavior
            fo.write(items[1] + "\t" + user + "\t" + movie_id + "\t" + cat1 +"\t" + mid_str + "\t" + cat_str+"\n")# label,uid,itemid,cate,itemid串起来，cate串起来
    last_user = user
    if clk:
        movie_id_list.append(movie_id)
        cate1_list.append(cat1)                
    line_idx += 1

## split_by_user.py

## generate_voc.py

In [46]:
path = 'D:/workspace/dien/data/cat_voc.pkl'
df = open(path,'rb')
file_pkl = pickle.load(df)

In [47]:
file_pkl


{'Residential': 1281,
 'Poetry': 250,
 'Winter Sports': 1390,
 'Olympics': 1128,
 'Body Art & Tattoo': 947,
 'Casseroles': 451,
 'Missions & Missionary Work': 964,
 'Home Care': 1545,
 'Extreme Sports': 772,
 'Mental Illness': 454,
 'Western': 905,
 'Authors': 192,
 'Experimental Psychology': 551,
 'Mystery': 427,
 'Arctic & Antarctica': 809,
 'Military': 55,
 'Diseases & Physical Ailments': 436,
 'Reference': 34,
 'Networking & System Administration': 899,
 'Educators': 605,
 'Natural Disasters': 798,
 'Early Childhood Education': 311,
 'Diabetic & Sugar-Free': 153,
 'Business Mathematics': 1214,
 'Business & Professional Growth': 1023,
 'Conventional': 805,
 'Embroidery': 1005,
 'Mystery & Wonders': 929,
 'Violence in Society': 806,
 'Agnosticism': 567,
 'Hockey': 1503,
 'Culture': 965,
 'Fish & Seafood': 575,
 'Interior Design': 592,
 'Game': 1054,
 'Low Salt': 491,
 'Women & Business': 619,
 'Budget': 198,
 'Scandinavian': 1546,
 'GRE': 1362,
 'Crocheting': 177,
 'Internet & Web Cu

In [49]:
path = 'D:/workspace/dien/data/mid_voc.pkl'
df = open(path,'rb')
mid_pkl = pickle.load(df)
mid_pkl

{'1594483752': 47396,
 '0738700797': 159716,
 '1439110239': 193476,
 '0768240018': 296682,
 '1572243678': 280207,
 '0970044208': 360954,
 'B00G7067FY': 83070,
 '0399250824': 126090,
 'B00D72WW7W': 106697,
 '0375709363': 296683,
 '1590770218': 244057,
 '0061189227': 257376,
 '014015941X': 255061,
 '0385334338': 6828,
 'B003AQBBQS': 166334,
 '0373797346': 202822,
 '0375833641': 19702,
 'B008JHVOHE': 248333,
 'B00AFFQ5G8': 47397,
 '1440228698': 341554,
 'B00J59Y1TS': 341555,
 '1466464445': 47398,
 'B007VCPTTW': 327967,
 '042523388X': 166335,
 '0761116982': 166336,
 '0743462823': 257377,
 '1456336185': 357663,
 '0147509742': 73462,
 'B00H5TD038': 35211,
 '1401206239': 341556,
 'B00AGV6D5E': 159717,
 'B008KBJIMW': 174968,
 '0316036358': 42305,
 '0312199406': 110202,
 'B00HMSXEO2': 75222,
 '0821761994': 280208,
 '0765312557': 51871,
 '0312388209': 135599,
 'B0052ABR2I': 140624,
 '0140437983': 257378,
 'B00959HOGA': 257379,
 'B00F9CEBXQ': 52807,
 '0590453858': 257380,
 'B0052BPKPC': 296684,
 

In [50]:
path = 'D:/workspace/dien/data/uid_voc.pkl'
df = open(path,'rb')
uid_pkl = pickle.load(df)
uid_pkl

{'ASEARD9XL1EWO': 449136,
 'AZPJ9LUT0FEPY': 0,
 'A2NRV79GKAU726': 16,
 'A2GEQVDX2LL4V3': 266686,
 'A3R04FKEYE19T6': 354817,
 'A3VGDQOR56W6KZ': 4,
 'A224BZLBXYJ3R7': 5,
 'ARPBBAKRFMD9U': 6,
 'AAJU4SG0NX7P': 7,
 'A54KRCUF41TZC': 447349,
 'AAM6CE0S73J6F': 10,
 'A3VRRTW5G76YXW': 241904,
 'A3JXCPTFBFAO9W': 11,
 'A301GPQ91DQ4XZ': 447351,
 'A2X2XR54RZUJ0M': 13,
 'A1SF55HOOP1T0Y': 14,
 'A3OCEYYZM0JLAZ': 15,
 'A2V9YBQYNRE48J': 1,
 'A3U03L03I1XNZW': 17,
 'A2XLNJ0BP4DKN6': 417286,
 'A3PF050ARC89QY': 452467,
 'A3D9KPMHSKHRC6': 18,
 'A2UQJ83KCGB6OC': 19,
 'A3IKY6O07XOCTF': 20,
 'A2YD10CM92JMO1': 21,
 'AKZ41HGB79YWY': 22,
 'ACS9O6A9BNZ9Z': 23,
 'A2R4BU3S05BGQU': 24,
 'A2S2PUF4OVD6CB': 25,
 'AJ2FDNZ3COBFN': 354822,
 'AEBNPOITGVN48': 477420,
 'AK45HQ9CHUWZ6': 27,
 'A1UNCTOLKTO3R6': 28,
 'A28ZV8CNFAOWAI': 29,
 'A1GX3G493YG1FL': 354823,
 'A23R100B5B81DY': 384176,
 'A32W6PTQPAZ6J0': 31,
 'A3G90QI891SKDP': 32,
 'A1CZKB5CLECA0P': 33,
 'A3KVXTYZLAMH2N': 34,
 'A18AQ3EFPYBKH0': 447360,
 'A1W0FB8CLWXV4X': 36,


In [51]:
len(uid_pkl)

543060

In [52]:
max(uid_pkl.values())

543059

# DataIterrator

In [41]:
import numpy
import json
import pickle as pkl
import random

import gzip

#import shuffle

def unicode_to_utf8(d):
    return dict((key.encode("UTF-8"), value) for (key,value) in d.items())

def load_dict(filename):
    try:
        with open(filename, 'rb') as f:
            return unicode_to_utf8(json.load(f))
    except:
        with open(filename, 'rb') as f:
            #return unicode_to_utf8(pkl.load(f))
            return pkl.load(f)


def fopen(filename, mode='r'):
    if filename.endswith('.gz'):
        return gzip.open(filename, mode)
    return open(filename, mode)

In [42]:
class DataIterator:

    def __init__(self, source,
                 uid_voc,
                 mid_voc,
                 cat_voc,
                 batch_size=128,
                 maxlen=100,
                 skip_empty=False,
                 shuffle_each_epoch=False,
                 sort_by_length=True,
                 max_batch_size=20,
                 minlen=None):
        if shuffle_each_epoch:
            self.source_orig = source
            self.source = shuffle.main(self.source_orig, temporary=True)
        else:
            self.source = fopen(source, 'r')
        self.source_dicts = []
        for source_dict in [uid_voc, mid_voc, cat_voc]:
            self.source_dicts.append(load_dict(source_dict))

        f_meta = open("D:/workspace/dien/data2/item-info", "r")
        meta_map = {}
        for line in f_meta:
            arr = line.strip().split("\t")
            if arr[0] not in meta_map:
                meta_map[arr[0]] = arr[1]
        self.meta_id_map ={}
        for key in meta_map:
            val = meta_map[key]
            if key in self.source_dicts[1]:
                mid_idx = self.source_dicts[1][key]
            else:
                mid_idx = 0
            if val in self.source_dicts[2]:
                cat_idx = self.source_dicts[2][val]
            else:
                cat_idx = 0
            self.meta_id_map[mid_idx] = cat_idx
        print('self.meta_id_map',len(self.meta_id_map))
        f_review = open("D:/workspace/dien/data1/reviews-info", "r")
        self.mid_list_for_random = []
        for line in f_review:
            arr = line.strip().split("\t")
            tmp_idx = 0
            if arr[1] in self.source_dicts[1]:
                tmp_idx = self.source_dicts[1][arr[1]]
            self.mid_list_for_random.append(tmp_idx)
        print('mid_list_for_random',self.mid_list_for_random[:5])

        self.batch_size = batch_size
        self.maxlen = maxlen
        self.minlen = minlen
        self.skip_empty = skip_empty

        self.n_uid = len(self.source_dicts[0])
        self.n_mid = len(self.source_dicts[1])
        self.n_cat = len(self.source_dicts[2])

        self.shuffle = shuffle_each_epoch
        self.sort_by_length = sort_by_length

        self.source_buffer = []
        self.k = batch_size * max_batch_size

        self.end_of_data = False

    def get_n(self):
        return self.n_uid, self.n_mid, self.n_cat

    def __iter__(self):
        return self

    def reset(self):
        if self.shuffle:
            self.source= shuffle.main(self.source_orig, temporary=True)
        else:
            self.source.seek(0)

    def next(self):
        if self.end_of_data:
            self.end_of_data = False
            self.reset()
            raise StopIteration

        source = []
        target = []

        if len(self.source_buffer) == 0:
            for k_ in range(self.k):
                ss = self.source.readline()
                if ss == "":
                    break
                self.source_buffer.append(ss.strip("\n").split("\t"))  # label uid itemid cate itemid串起来 cate串起来

            # sort by  history behavior length
            if self.sort_by_length:
                his_length = numpy.array([len(s[4].split("")) for s in self.source_buffer])  # 分割itemid_str
                tidx = his_length.argsort()  # 升序排序，记录其idx

                _sbuf = [self.source_buffer[i] for i in tidx]
                self.source_buffer = _sbuf  # 排序
            else:
                self.source_buffer.reverse()

        if len(self.source_buffer) == 0:
            self.end_of_data = False
            self.reset()
            raise StopIteration

        try:
            cntcnt = 0
            # actual work here
            while True:
                cntcnt += 1
                print(cntcnt)
                # read from source file and map to word index
                try:
                    ss = self.source_buffer.pop()
                    print('ss',ss)
                except IndexError:
                    break
                
                print(ss[1],ss[1] in self.source_dicts[0])
                print(ss[2],ss[1] in self.source_dicts[1])
                print(ss[3],ss[1] in self.source_dicts[2])
                #print(self.source_dicts[0])
                
                uid = self.source_dicts[0][ss[1]] if ss[1] in self.source_dicts[0] else 0
                mid = self.source_dicts[1][ss[2]] if ss[2] in self.source_dicts[1] else 0
                cat = self.source_dicts[2][ss[3]] if ss[3] in self.source_dicts[2] else 0
                tmp = []
                for fea in ss[4].split(""):
                    m = self.source_dicts[1][fea] if fea in self.source_dicts[1] else 0
                    tmp.append(m)
                mid_list = tmp  # str转idx

                tmp1 = []
                for fea in ss[5].split(""):
                    c = self.source_dicts[2][fea] if fea in self.source_dicts[2] else 0
                    tmp1.append(c)
                cat_list = tmp1  # str转idx
                print('mid_list',mid_list,'cat_list',cat_list)
                # read from source file and map to word index

                #if len(mid_list) > self.maxlen:
                #    continue
                if self.minlen != None:
                    if len(mid_list) <= self.minlen:
                        print('continue1')
                        continue
                if self.skip_empty and (not mid_list):
                    print('continue1')
                    continue

                noclk_mid_list = []
                noclk_cat_list = []
                for pos_mid in mid_list:
                    
                    print('pos_mid',pos_mid)
                    noclk_tmp_mid = []
                    noclk_tmp_cat = []
                    noclk_index = 0
                    while True:
                        noclk_mid_indx = random.randint(0, len(self.mid_list_for_random)-1)
                        noclk_mid = self.mid_list_for_random[noclk_mid_indx]
                        #if noclk_mid == pos_mid:
                        #    print('noclk_mid',noclk_mid,'pos_mid',pos_mid)
                        #    continue
                        noclk_tmp_mid.append(noclk_mid)
                        noclk_tmp_cat.append(self.meta_id_map[noclk_mid])
                        noclk_index += 1
                        print('noclk_tmp_mid',noclk_tmp_mid)
                        print('noclk_tmp_cat',noclk_tmp_cat)
                        if noclk_index >= 5:
                            break
                    noclk_mid_list.append(noclk_tmp_mid)  # 每个真item_list中一个item配5个neg采样样本
                    noclk_cat_list.append(noclk_tmp_cat)  # 配套对应的负采样cate
                source.append([uid, mid, cat, mid_list, cat_list, noclk_mid_list, noclk_cat_list])
                target.append([float(ss[0]), 1-float(ss[0])])  # []
                print('cnt:{}'.format(cntcnt),source,target)
                if len(source) >= self.batch_size or len(target) >= self.batch_size:
                    break
        except IOError:
            self.end_of_data = True

        # all sentence pairs in maxibatch filtered out because of length
        if len(source) == 0 or len(target) == 0:
            source, target = self.next()

        return source, target

In [43]:
train_file = "D:/workspace/dien/data/local_train_splitByUser"
test_file = "D:/workspace/dien/data/local_test_splitByUser"
uid_voc = "D:/workspace/dien/data/uid_voc.pkl"
mid_voc = "D:/workspace/dien/data/mid_voc.pkl"
cat_voc = "D:/workspace/dien/data/cat_voc.pkl"
batch_size = 4
maxlen = 6
test_iter = 5
save_iter = 5

In [44]:
train_data=DataIterator(train_file, uid_voc, mid_voc, cat_voc, batch_size, maxlen, shuffle_each_epoch=False)

self.meta_id_map 367983
mid_list_for_random [4275, 4275, 4275, 4275, 4275]


In [45]:
train_data.get_n()  # num_uid, num_itemid, num_cate

(543060, 367983, 1601)

In [46]:
s,t=train_data.next()
s,t

1
ss ['1', 'AJ2FDNZ3COBFN', '1594488339', 'Books', '0525951792\x020141039280\x020143034901\x020060738871\x021400069017\x020545230500\x020470889284\x020399155767\x020679426310\x020307744566\x020062110837\x02054799219X\x020345805038\x020306821761\x020761458239\x020761461035\x020062293192\x020062088254\x020802121624\x02006228553X\x020345539788\x021590516400\x021594486093\x020812993411\x020399157727\x020544003454\x020547970447\x020778315479\x020857051873\x021401931758\x021453299335\x02038553762X\x02125001980X\x021565129903\x021477849815\x021477808647\x020062202235\x020307408973\x020449809978\x020062224735\x021455573205\x020062285505\x02030727179X\x020670015709\x020451418905\x020062279874\x020670016780\x02077831586X\x021423164911\x020307361047\x020345540700\x02039916748X\x020307266389\x020385536038\x020062103172\x021423171020\x020988767325\x021599907909\x021594631395\x020316254770\x020525426035\x020804138680\x020061713783\x020345536223\x020385537433\x021455547387\x020062246879\x020062291882

pos_mid 23647
noclk_tmp_mid [43058]
noclk_tmp_cat [1]
noclk_tmp_mid [43058, 91152]
noclk_tmp_cat [1, 1]
noclk_tmp_mid [43058, 91152, 3393]
noclk_tmp_cat [1, 1, 1]
noclk_tmp_mid [43058, 91152, 3393, 4597]
noclk_tmp_cat [1, 1, 1, 2]
noclk_tmp_mid [43058, 91152, 3393, 4597, 80272]
noclk_tmp_cat [1, 1, 1, 2, 1]
pos_mid 14919
noclk_tmp_mid [86220]
noclk_tmp_cat [1]
noclk_tmp_mid [86220, 6013]
noclk_tmp_cat [1, 1]
noclk_tmp_mid [86220, 6013, 29919]
noclk_tmp_cat [1, 1, 1]
noclk_tmp_mid [86220, 6013, 29919, 5936]
noclk_tmp_cat [1, 1, 1, 1]
noclk_tmp_mid [86220, 6013, 29919, 5936, 270037]
noclk_tmp_cat [1, 1, 1, 1, 1]
pos_mid 34478
noclk_tmp_mid [52026]
noclk_tmp_cat [1]
noclk_tmp_mid [52026, 132]
noclk_tmp_cat [1, 1]
noclk_tmp_mid [52026, 132, 19118]
noclk_tmp_cat [1, 1, 1]
noclk_tmp_mid [52026, 132, 19118, 25205]
noclk_tmp_cat [1, 1, 1, 5]
noclk_tmp_mid [52026, 132, 19118, 25205, 58096]
noclk_tmp_cat [1, 1, 1, 5, 1]
pos_mid 14010
noclk_tmp_mid [74812]
noclk_tmp_cat [1]
noclk_tmp_mid [74812, 

noclk_tmp_cat [1, 1, 1]
noclk_tmp_mid [98644, 236509, 32573, 127186]
noclk_tmp_cat [1, 1, 1, 46]
noclk_tmp_mid [98644, 236509, 32573, 127186, 352281]
noclk_tmp_cat [1, 1, 1, 46, 1]
pos_mid 78130
noclk_tmp_mid [53488]
noclk_tmp_cat [1]
noclk_tmp_mid [53488, 223405]
noclk_tmp_cat [1, 1]
noclk_tmp_mid [53488, 223405, 1365]
noclk_tmp_cat [1, 1, 1]
noclk_tmp_mid [53488, 223405, 1365, 17269]
noclk_tmp_cat [1, 1, 1, 1]
noclk_tmp_mid [53488, 223405, 1365, 17269, 2883]
noclk_tmp_cat [1, 1, 1, 1, 1]
pos_mid 102197
noclk_tmp_mid [2409]
noclk_tmp_cat [1]
noclk_tmp_mid [2409, 5779]
noclk_tmp_cat [1, 1]
noclk_tmp_mid [2409, 5779, 5464]
noclk_tmp_cat [1, 1, 1]
noclk_tmp_mid [2409, 5779, 5464, 171061]
noclk_tmp_cat [1, 1, 1, 1]
noclk_tmp_mid [2409, 5779, 5464, 171061, 94397]
noclk_tmp_cat [1, 1, 1, 1, 1]
pos_mid 208848
noclk_tmp_mid [32582]
noclk_tmp_cat [1]
noclk_tmp_mid [32582, 52475]
noclk_tmp_cat [1, 1]
noclk_tmp_mid [32582, 52475, 321513]
noclk_tmp_cat [1, 1, 1]
noclk_tmp_mid [32582, 52475, 32151

([[354822,
   247491,
   1,
   [30759,
    9,
    397,
    22470,
    33267,
    4345,
    107079,
    17437,
    92199,
    9840,
    430,
    274,
    25418,
    33588,
    64902,
    58509,
    32832,
    9248,
    15152,
    7782,
    1946,
    29504,
    44531,
    44652,
    40259,
    73786,
    76003,
    27074,
    5815,
    16591,
    51322,
    11951,
    3787,
    5658,
    43278,
    2359,
    43783,
    12085,
    9432,
    4293,
    18249,
    18277,
    14071,
    37511,
    37336,
    6635,
    16083,
    33010,
    11738,
    16331,
    27608,
    197892,
    40977,
    23024,
    43178,
    3085,
    79727,
    68223,
    17434,
    20789,
    18569,
    20281,
    23647,
    14919,
    34478,
    14010,
    59385,
    44327,
    23947,
    4515,
    50301,
    62648,
    52002,
    54416,
    33253,
    5542,
    53119,
    77959,
    33996],
   [1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,

In [52]:
len(s[0][4])

79

In [54]:
len(s[0][5])

79

In [56]:
len(s[0][6])

79

In [14]:
t

[[1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0]]

In [57]:
def prepare_data(input, target, maxlen = None, return_neg = False):
    # x: a list of sentences
    lengths_x = [len(s[4]) for s in input]  # uid点击的item hist
    seqs_mid = [inp[3] for inp in input]
    seqs_cat = [inp[4] for inp in input]
    noclk_seqs_mid = [inp[5] for inp in input]
    noclk_seqs_cat = [inp[6] for inp in input]

    if maxlen is not None:
        new_seqs_mid = []
        new_seqs_cat = []
        new_noclk_seqs_mid = []
        new_noclk_seqs_cat = []
        new_lengths_x = []
        for l_x, inp in zip(lengths_x, input):
            if l_x > maxlen:
                new_seqs_mid.append(inp[3][l_x - maxlen:])
                new_seqs_cat.append(inp[4][l_x - maxlen:])
                new_noclk_seqs_mid.append(inp[5][l_x - maxlen:])
                new_noclk_seqs_cat.append(inp[6][l_x - maxlen:])
                new_lengths_x.append(maxlen)
            else:
                new_seqs_mid.append(inp[3])
                new_seqs_cat.append(inp[4])
                new_noclk_seqs_mid.append(inp[5])
                new_noclk_seqs_cat.append(inp[6])
                new_lengths_x.append(l_x)
        lengths_x = new_lengths_x
        seqs_mid = new_seqs_mid
        seqs_cat = new_seqs_cat
        noclk_seqs_mid = new_noclk_seqs_mid
        noclk_seqs_cat = new_noclk_seqs_cat

        if len(lengths_x) < 1:
            return None, None, None, None

    n_samples = len(seqs_mid)
    maxlen_x = numpy.max(lengths_x)
    neg_samples = len(noclk_seqs_mid[0][0])

    mid_his = numpy.zeros((n_samples, maxlen_x)).astype('int64')
    cat_his = numpy.zeros((n_samples, maxlen_x)).astype('int64')
    noclk_mid_his = numpy.zeros((n_samples, maxlen_x, neg_samples)).astype('int64')
    noclk_cat_his = numpy.zeros((n_samples, maxlen_x, neg_samples)).astype('int64')
    mid_mask = numpy.zeros((n_samples, maxlen_x)).astype('float32')
    for idx, [s_x, s_y, no_sx, no_sy] in enumerate(zip(seqs_mid, seqs_cat, noclk_seqs_mid, noclk_seqs_cat)):
        mid_mask[idx, :lengths_x[idx]] = 1.
        mid_his[idx, :lengths_x[idx]] = s_x
        cat_his[idx, :lengths_x[idx]] = s_y
        noclk_mid_his[idx, :lengths_x[idx], :] = no_sx
        noclk_cat_his[idx, :lengths_x[idx], :] = no_sy

    uids = numpy.array([inp[0] for inp in input])
    mids = numpy.array([inp[1] for inp in input])
    cats = numpy.array([inp[2] for inp in input])

    if return_neg:
        return uids, mids, cats, mid_his, cat_his, mid_mask, numpy.array(target), numpy.array(lengths_x), noclk_mid_his, noclk_cat_his

    else:
        return uids, mids, cats, mid_his, cat_his, mid_mask, numpy.array(target), numpy.array(lengths_x)

In [58]:
uids, mids, cats, mid_his, cat_his, mid_mask, target, sl, noclk_mids, noclk_cats = prepare_data(s, t, maxlen=100, return_neg=True)

In [74]:
sl

array([79, 79, 31, 31])

In [60]:
mids

array([247491,  43549, 194694,   7610])

In [61]:
cats

array([1, 1, 2, 1])

In [72]:
mid_his

array([[ 30759,      9,    397,  22470,  33267,   4345, 107079,  17437,
         92199,   9840,    430,    274,  25418,  33588,  64902,  58509,
         32832,   9248,  15152,   7782,   1946,  29504,  44531,  44652,
         40259,  73786,  76003,  27074,   5815,  16591,  51322,  11951,
          3787,   5658,  43278,   2359,  43783,  12085,   9432,   4293,
         18249,  18277,  14071,  37511,  37336,   6635,  16083,  33010,
         11738,  16331,  27608, 197892,  40977,  23024,  43178,   3085,
         79727,  68223,  17434,  20789,  18569,  20281,  23647,  14919,
         34478,  14010,  59385,  44327,  23947,   4515,  50301,  62648,
         52002,  54416,  33253,   5542,  53119,  77959,  33996],
       [ 30759,      9,    397,  22470,  33267,   4345, 107079,  17437,
         92199,   9840,    430,    274,  25418,  33588,  64902,  58509,
         32832,   9248,  15152,   7782,   1946,  29504,  44531,  44652,
         40259,  73786,  76003,  27074,   5815,  16591,  51322,  11951,

In [78]:
mid_mask[0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)

In [79]:
noclk_mids[0]

array([[202026, 195112,  90804,  25926, 349341],
       [ 40668,   2219,    298,  48860,  55522],
       [100698,  30918, 224100, 195749,    469],
       [  2249,     28,  79268,  85281,   5722],
       [  4580,   1271,   2094,  14072, 274906],
       [  3005,  70042,   3275,  16283,  38053],
       [  3529,  30283,   6291,   6305,  35080],
       [232539,   6560,   6144, 157057,  42404],
       [ 45858,  64028,   3466,  39036,  90369],
       [ 30981,  32788,  15364,   1177,   3876],
       [   148,  48453,  29201, 357798,  49170],
       [ 52560,    177,   2872, 121841,    325],
       [  2307,  16474,  53235, 281956,   3347],
       [ 63598,  23117,    293,   1380,  23761],
       [   213, 110695, 196476,  21833,  12183],
       [177745, 232206,  44117,  29895,  12242],
       [   667,  71734, 320149,  12486,  64297],
       [  1030, 312376,   5277, 115346, 190257],
       [    18,   1086,  28902, 207268,  27678],
       [ 72595, 202323,   6094,  27030,   3140],
       [  8732,   16